In [1]:
"""
Created on Sat Oct 28 16:16:39 2017

@author: 麦芽的香气
"""
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb
#import lightgbm as lgb    

In [2]:
path='./data/'
df=pd.read_csv(path+u'训练数据-ccf_first_round_user_shop_behavior.csv')
shop=pd.read_csv(path+u'训练数据-ccf_first_round_shop_info.csv')
test=pd.read_csv(path+u'AB榜测试集-evaluation_public.csv')
df.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


In [3]:
df=pd.merge(df,shop[['shop_id','mall_id']],how='left',on='shop_id')
#left: use only keys from left frame, similar to a SQL left outer join; preserve key order
#Field names to join on. Must be found in both DataFrames. If on is None and not merging on indexes, then it merges on the intersection of the columns by default.
df['time_stamp']=pd.to_datetime(df['time_stamp'])
#Convert argument to datetime.
train=pd.concat([df,test])
print(df.shape,test.shape,train.shape)
mall_list=list(set(list(shop.mall_id)))
result=pd.DataFrame()

(1138015, 7) (483931, 7) (1621946, 8)


In [4]:
df.head()

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,mall_id
0,u_376,s_2871718,2017-08-06 21:20:00,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,m_1409
1,u_376,s_2871718,2017-08-06 21:20:00,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,m_1409
2,u_1041,s_181637,2017-08-02 13:10:00,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,m_4079
3,u_1158,s_609470,2017-08-13 12:30:00,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,m_6587
4,u_1654,s_3816766,2017-08-25 19:50:00,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,m_3005


In [9]:
for mall in mall_list:
    train1=train[train.mall_id==mall].reset_index(drop=True)     
    #reset_index:新增一列从0开始的索引，drop为true则去掉旧索引

    #统计了不同wifi出现次数，以及把每个wifi的值添加到每一行
    l=[]
    wifi_dict = {}
    for index,row in train1.iterrows():
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        for i in wifi_list:
            row[i[0]]=int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]]=1
            else:
                wifi_dict[i[0]]+=1
            #wifi_dict[i[0]] = wifi_dict.get(i[0],1)+1
        l.append(row)  
    
    delate_wifi=[] #统计出现次数小于20的
    for i in wifi_dict:
        if wifi_dict[i]<20:
            delate_wifi.append(i)
            
    #每一行为一条记录的wifi值 字典，k为次数不小于20的wifiid，v为其值大小
    m=[]
    for row in l:
        new={}
        for n in row.keys(): #keys相当于一行的每一列的列名
            if n not in delate_wifi:  #这里也包含了wifi之外的信息比如lola等
                new[n]=row[n]
        m.append(new)
        
    train1=pd.DataFrame(m) #m每一行为一个字典 转df后 相同值为一列，没有的就为nan

    #一起处理完后再分开
    df_train=train1[train1.shop_id.notnull()]
    df_test=train1[train1.shop_id.isnull()]
    
    lbl = preprocessing.LabelEncoder()  #LabelEncoder就是按照从0到len()重新编号
    lbl.fit(list(df_train['shop_id'].values))  #fit进set值
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    #再transform待转换的数据 
    num_class=df_train['label'].max()+1    #这里就得到了有多少个shop
    
    params = {
            'objective': 'multi:softmax',
            'eta': 0.1,
            'max_depth': 9,
            'eval_metric': 'merror',
            'seed': 0,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]
    #DF.cloumns 遍历列名 取得特征列的列名

    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ] #这里相当于没划分验证集
    num_rounds=60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    #Will train until test-merror hasn't improved in 15 rounds.
    
    #predict
    df_test['label']=model.predict(xgbtest)
    df_test['shop_id']=df_test['label'].apply(lambda x:lbl.inverse_transform(int(x)))
    r=df_test[['row_id','shop_id']]
    result=pd.concat([result,r]) #在循环外用一个result 这样每训练完一个mall就cancat，然后覆盖保存
    result['row_id']=result['row_id'].astype('int')
    result.to_csv(path+'sub.csv',index=False)

f:\software\python\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.106659	test-merror:0.106659
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.086788	test-merror:0.086788
[2]	train-merror:0.075602	test-merror:0.075602
[3]	train-merror:0.069022	test-merror:0.069022
[4]	train-merror:0.062837	test-merror:0.062837
[5]	train-merror:0.05731	test-merror:0.05731
[6]	train-merror:0.052836	test-merror:0.052836
[7]	train-merror:0.047967	test-merror:0.047967
[8]	train-merror:0.044677	test-merror:0.044677
[9]	train-merror:0.041321	test-merror:0.041321
[10]	train-merror:0.038426	test-merror:0.038426
[11]	train-merror:0.036189	test-merror:0.036189
[12]	train-merror:0.034018	test-merror:0.034018
[13]	train-merror:0.032438	test-merror:0.032438
[14]	train-merror:0.030399	test-merror:0.030399
[15]	train-merror:0.029214	test-merror:0.029214
[16]	train-merror:0.027306	test-merror:0.027306
[17]	train-merror:0.026122	test-merror:0.026122
[

f:\software\python\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1


f:\software\python\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
